<a href="https://colab.research.google.com/github/cheng1103/cheng1103/blob/main/Spider_cnyes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><strong>備忘錄：</strong></p>

<p>鉅亨網新聞資料起始日：2009/05/30</p>


In [ ]:
#https://colab.research.google.com/drive/12CvYz3OMLgMl1dVW69KEO5IhWgsqUJCj#scrollTo=QvlG2w58eYLV

#透過 ckiptagger 來使用繁體中文斷詞的最佳工具 CKIP
#https://clay-atlas.com/blog/2019/09/24/python-chinese-tutorial-ckiptagger/
#GKIPtagger – 國產斷詞工具
#http://13.231.129.69/2020/12/31/python-gkiptagger-%E5%9C%8B%E7%94%A2%E6%96%B7%E8%A9%9E%E5%B7%A5%E5%85%B7/
#中文NLP
#https://medium.com/%E4%B8%AD%E6%96%87-nlp-%E8%99%95%E7%90%86

#鉅亨網區塊鏈
#

In [ ]:
#安裝套件
#!pip install ckipnlp






#==============================================
#清除所有變數
%reset -f

# **懶惰蟲：自動點擊程式(適用在Edge與chrome)**

In [ ]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}
setInterval(ClickConnect,Math.random()*1000*60*3)
'''

# **安裝並載入套件**

In [ ]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd


#https://api.cnyes.com/media/api/v1/newslist/all?startAt=1642608000&endAt=1643558399&limit=30
#https://api.cnyes.com/media/api/v1/newslist/category/headline?startAt=1642608000&endAt=1643558399&limit=30

# **爬取鉅亨網新聞網址清單**

In [ ]:
def url_list(s_time,e_time):
  
  #==============================================
  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800

  #==============================================
  p = 1
  l_page = 9
  data = pd.DataFrame()

  while p <= l_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      l_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp_data = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp_data['publishAt']
    newsId = temp_data['newsId']
    title = temp_data['title']
    #isIndex = temp_data['isIndex'] 
    isCategoryHeadline = temp_data['isCategoryHeadline']
    #categoryId = temp_data['categoryId']
    
    temp_data = pd.concat([publishAt, newsId, title, isCategoryHeadline ], 1)
    data = pd.concat([data, temp_data])
    
    

    #休眠
    t = random.uniform(0.005, 0.25) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%50 == 0:
      print(p/l_page*100)
      time.sleep(0.5)

    if p%200 == 0:
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      time.sleep(2)

  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
  return(data)



# **新聞類別與區間設定**

In [ ]:
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#查詢區間
s_time = "2021-01-01 00:00:00" #開始日期 
e_time = "2021-04-01 00:00:00" #結束日期


#==============================================
from fake_useragent import UserAgent
data_URL = []

kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

**執行程式**

In [ ]:
data_URL = url_list(s_time,e_time)
data_URL = data_URL.sort_values(by='publishAt', ascending=True)
data_URL = data_URL.reset_index(drop=True) 

37.03703703703704
74.07407407407408


In [ ]:
#檢視資料
data_URL

,publishAt,newsId,title,isCategoryHeadline
0,1609472787,4555666,〈潛力股〉受惠車市快速復甦及電動車產業起飛 敬鵬營運不淡,1
1,1609472921,4555657,〈潛力股〉和大搭電動車成長順風車 獲利動能加溫,1
2,1609473015,4555641,〈潛力股〉閎暉調整營運體質見效 明年挑戰擺脫連5年虧損,1
3,1609473064,4555695,〈潛力股〉環宇-KY基站回溫、轉投資發酵 營運拚反彈,1
4,1609473391,4555707,〈潛力股〉奇力新電感、LTCC需求展望佳 併帛漢挹注長期獲利動能,1
...,...,...,...,...
4028,1617280223,4623303,LCD及DRAM輪流大漲 透露Q2選股方向,1
4029,1617280816,4623477,〈台積電大擴產〉砸千億美元擴廠 設備業者迎高速成長期,1
4030,1617281688,4623507,壽險、券商前2月減碼台股逾610億元 金融三業可加碼銀彈逾8.3兆元,1
4031,1617282099,4623513,改列全額交割股 台開：將加速處分資產並取得融資協助,1


In [ ]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd

data = pd.DataFrame()
l_page = len(data_URL)

for i in range(l_page):

  headers = UserAgent().random
  url = data_URL['newsId'][i]
  url = 'https://news.cnyes.com/news/id/'+str(url)

  req = requests.get(url, headers={"User-Agent":headers})
  req = req.content.decode()
  req = etree.HTML(req)

  catalog = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[1]/nav/a/span/span/text()')
  catalog = ",".join(catalog)

  title = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/h1/text()')
  title = "".join(title)

  reporter = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/span/span/text()')
  reporter = "".join(reporter)


  d_time = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/time/text()')
  d_time = "".join(d_time)

  content = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/text() | //*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/a/text()')
  content = "".join(content)

  tag = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/nav/a/span/text()')
  tag = ",".join(tag)

  publishAt = data_URL['publishAt'][i]
  #publishAt = url_list['publishAt'](index)


  temp_data ={(publishAt ,d_time, catalog, title, reporter, content, tag, url)}
  temp_data = pd.DataFrame(temp_data)
  #temp_data.columns = ['time', 'publishAt', 'catalog', 'title', 'reporter', 'content', 'tag']
  data = data.append(temp_data)

  t = random.uniform(0.005, 0.2) #隨機數
  time.sleep(t) #暫停

  if i % 50 == 0:
    print(i/l_page*100)
    time.sleep(0.5)

  if i % 250 == 0:
    time.sleep(2.5)



0.0
1.239771881973717
2.479543763947434
3.7193156459211507
4.959087527894868
6.198859409868584
7.438631291842301
8.678403173816019
9.918175055789735
11.157946937763452
12.397718819737168
13.637490701710886
14.877262583684603
16.11703446565832
17.356806347632038
18.596578229605754
19.83635011157947
21.076121993553187
22.315893875526903
23.55566575750062
24.795437639474336
26.035209521448056
27.274981403421773
28.51475328539549
29.754525167369206


In [ ]:
data.columns = [ 'publishAt', 'time', 'catalog', 'title', 'reporter', 'content', 'tag', 'url']

data

'''
from google.colab import drive
drive.mount('/content/drive')
'''


#輸出檔案
#data.to_csv('/content/drive/MyDrive/Colab Notebooks/sample.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#檔案讀寫
'''
from google.colab import drive
drive.mount('/content/drive')
'''
'''
#輸出檔案
data_url.to_csv('/content/drive/MyDrive/Colab Notebooks/cnyes-url-2017.csv')



url = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cnyes-url-2020.csv')


'''

#url = url.drop(columns=['Unnamed: 0'])